<a href="https://colab.research.google.com/github/tina589/machine-learning-bootcamp-1/blob/main/supervised%20/01_basics/03_feature_exctraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import bibliotek**

In [1]:
import numpy as p
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

**Załadowanie danych**

In [2]:
def fetch_financial_data (company='AMZN'):  #domyślnie pobierze notowania spółki amazon (amzn) - ticker (mozna znaleźć inne w bliotece, tu korzystamy ze żródła danych stooq)
  """
  This function fetches stock market quotations.
  """
  import pandas_datareader.data as web
  return web.DataReader(name=company, data_source='stooq')

df_raw = fetch_financial_data()
df_raw.head()  #wyświetla tylko 5 pierwszych wierszy

,Open,High,Low,Close,Volume
Date,,,,,
2023-08-02,130.154,130.23,126.8200,128.21,51027614
2023-08-01,133.550,133.69,131.6199,131.69,42298925
2023-07-31,133.200,133.87,132.3800,133.68,41901516
2023-07-28,129.690,133.01,129.3300,132.21,46317381
2023-07-27,131.000,132.63,127.7900,128.25,52610661


**Utworzenie kopii danych**

In [3]:
df = df_raw.copy()
df = df[:5]  #wyciecie 5 pierwszych wierszy
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2023-08-02 to 2023-07-27
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


**Generowanie nowych zmiennych**

In [4]:
df['day']=df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2023-08-02,130.154,130.23,126.8200,128.21,51027614,2,8,2023
2023-08-01,133.550,133.69,131.6199,131.69,42298925,1,8,2023
2023-07-31,133.200,133.87,132.3800,133.68,41901516,31,7,2023
2023-07-28,129.690,133.01,129.3300,132.21,46317381,28,7,2023
2023-07-27,131.000,132.63,127.7900,128.25,52610661,27,7,2023


**Dyskretyzacja zmiennej ciągłej**

In [5]:
df = pd.DataFrame(data={'height':[175., 185.5, 185., 191., 184.5, 183., 168.] }) #generowanie przykładowych danych
df

,height
0,175.0
1,185.5
2,185.0
3,191.0
4,184.5
5,183.0
6,168.0


In [6]:
df['height_cat'] = pd.cut(x=df.height, bins = 3) #bins - na ile podziałów podzielić zmienną ciągłą
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,185.5,"(183.333, 191.0]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


In [7]:
df['height_cat'] = pd.cut(x=df.height, bins = (160, 175, 180, 195)) #podanie konkretnych przedziałów
df

,height,height_cat
0,175.0,"(160, 175]"
1,185.5,"(180, 195]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


In [8]:
df['height_cat'] = pd.cut(x=df.height, bins = (160, 175, 180, 195), labels = ['small', 'medium', 'high']) #dodanie konkretnej etykiety
df

,height,height_cat
0,175.0,small
1,185.5,high
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


In [10]:
pd.get_dummies(df,drop_first=True, prefix='height') #przygotowanie danych do modelu

,height,height_medium,height_high
0,175.0,0,0
1,185.5,0,1
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,183.0,0,1
6,168.0,0,0


**Ekstrakcja cech**

In [11]:
df = pd.DataFrame(data = {'lang':[['PL','EN'], ['GER','ENG','PL','FRA'], ['RUS']]}) #generowanie danych
df

,lang
0,"[PL, EN]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


In [12]:
df['lang_number'] = df['lang'].apply(len) #wydobycie zmiennej pokazującej liczbe elementów (języków), len - zwraca długość listy
df

,lang,lang_number
0,"[PL, EN]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


In [13]:
df['PL_flag'] = df['lang'].apply(lambda x:1 if 'PL' in x else 0) #zbudowanie flagi, czy język polski występuje w liście
df

,lang,lang_number,PL_flag
0,"[PL, EN]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


In [14]:
df = pd.DataFrame(data={'website':['wp.pl','onet.pl','google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [15]:
df.website.str.split('.',expand = True) #podział stringa w miejscu kropki

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [16]:
new = df.website.str.split('.', expand = True)
df['portal'] = new[0]
df['extension'] = new[1]
df

,website,portal,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
